# TODO :  change from sorting by resource type to profile

## Create FHIR R4 SearchParameter Resource

Create FHIR R4 SearchParameter Resource, Quick start text, and Searchparameter list using the python fhir client

Source data is in excel file

### Prerequisites:

- Python 3.7 or greater


### Import FHIRR4Client and other libraries

In [ ]:
%config IPCompleter.greedy=True

In [ ]:
from fhirclient.r4models.fhirabstractbase import FHIRValidationError
from fhirclient.r4models import searchparameter as SP
from fhirclient.r4models import capabilitystatement as CS
from fhirclient.r4models import bundle as B
import fhirclient.r4models.identifier as I
import fhirclient.r4models.coding as C
import fhirclient.r4models.codeableconcept as CC
import fhirclient.r4models.fhirdate as D
import fhirclient.r4models.extension as X
import fhirclient.r4models.contactdetail as CD
from json import dumps, loads, load
from requests import get, post, put
import os
from pathlib import Path
from csv import reader as csvreader
from IPython.display import display as Display, HTML, Markdown
from pprint import pprint
from collections import namedtuple
from pandas import *
from datetime import datetime
from jinja2 import Environment, FileSystemLoader, select_autoescape
import R4sp_summary_list as sp_map
from stringcase import snakecase, titlecase, pascalcase
from itertools import zip_longest
from openpyxl import load_workbook

####  Assign Global Variables


Here is where we assign all the global variables for this example such as the local paths for file input and output

##### Need to update:
- base_id
- paths
- canonical

In [ ]:
#******************** Need to update *************************************************
fhir_base_url = 'http://hl7.org/fhir/'
base_id = "US-Core"
canon_base = "http://hl7.org/fhir/us/core/"
ig_folder = 'US-Core'
publisher = 'HL7 International - Structured Documents Work Group'
publisher_endpoint = dict(
                        system = 'url',
                        value = 'http://www.hl7.org/Special/committees/structure/index.cfm'
                        ) 
ig_source_path = "//ERICS-AIR-2/ehaas/Documents/FHIR/US-Core-R4/source/"
#ig_source_path = "/Users/ehaas/Documents/FHIR/US-Core-R4/source/"
#ig_source_path = ''
ig_pages_path= '//ERICS-AIR-2/ehaas/Documents/FHIR/US-Core-R4/source/pages/_includes'
#ig_pages_path= '/Users/ehaas/Documents/FHIR/US-Core-R4/source/pages/_includes'
#ig_pages_path = ''

spdef_json = 'C:/Users/Eric/Documents/HL7/FHIR/BUILD_EDIT_FILES/R4_Definitions/search-parameters.json'
#spdef_json ='/Users/ehaas/Downloads/definitionsR4.json/search-parameters.json'

skip_types = ['Questionnaire']
#***********************************************************************************

env = Environment(
    loader=FileSystemLoader(searchpath = ''),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )

md_template = ['quick_start.j2', 'sp_list_page.j2', 'cs_search_documentation.j2','sp_narrative.j2']


fhir_term_server = 'http://test.fhir.org/r3'

# profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient



none_list = ['', ' ', 'none', 'n/a', 'N/A', 'N', 'False', 'FALSE']
sep_list = (',', ';', ' ', ', ', '; ')



### validate

In [ ]:
# *********************** validate Resource as Dict ********************************

def validate(r):

    fhir_test_server = 'http://test.fhir.org/r4'

    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }
    
    params = {
    }
    
    r = post(f'{fhir_test_server}/{r["resourceType"]}/$validate', params = params, headers = headers, data = dumps(r))
    # return r.status_code
    # view  output
    # return (r.json()["text"]["div"])
    return r

### Get Search Parameter input data

In [29]:
#in_path = '/Users/ehaas/Documents/FHIR/pyfhir/test/'
in_path =''

in_file ="uscore-server"

xls = ExcelFile(f'{in_path}{in_file}.xlsx')
df = read_excel(xls,'sps',na_filter = False)
df_combos = read_excel(xls,'sp_combos',na_filter = False)

df_combos

,index,base,profile,combo,combo_conf,types,fixed_kv,description,example,imp_note
0,1,!Encounter,http://hl7.org/fhir/us/core/StructureDefinitio...,"class,date",SHOULD,"date,token",,,,Fetches a bundle of all !Encounter resources m...
1,2,!Encounter,http://hl7.org/fhir/us/core/StructureDefinitio...,"class,date,patient",SHOULD,"date,reference,token",,,,Fetches a bundle of all !Encounter resources m...
2,3,!Encounter,http://hl7.org/fhir/us/core/StructureDefinitio...,"class,date,patient,type",SHOULD,"date,reference,token",,,,Fetches a bundle of all !Encounter resources m...
3,4,!Encounter,http://hl7.org/fhir/us/core/StructureDefinitio...,"class,date,type",SHOULD,"date,token",,,,Fetches a bundle of all !Encounter resources m...
4,5,Encounter,http://hl7.org/fhir/us/core/StructureDefinitio...,"class,patient",SHOULD,"reference,token",,support searching for all encounter for a pati...,GET [base]/Encounter?patient=example1&class= h...,Fetches a bundle of all Encounter resources ma...
5,6,!Encounter,http://hl7.org/fhir/us/core/StructureDefinitio...,"class,patient,status",SHOULD,"reference,token",,,,Fetches a bundle of all !Encounter resources m...
6,7,!Encounter,http://hl7.org/fhir/us/core/StructureDefinitio...,"class,patient,status,type",SHOULD,"reference,token",,,,Fetches a bundle of all !Encounter resources m...
7,8,!Encounter,http://hl7.org/fhir/us/core/StructureDefinitio...,"class,patient,type",SHOULD,"reference,token",,,,Fetches a bundle of all !Encounter resources m...
8,9,!Encounter,http://hl7.org/fhir/us/core/StructureDefinitio...,"class,status",SHOULD,token,,,,Fetches a bundle of all !Encounter resources m...
9,10,!Encounter,http://hl7.org/fhir/us/core/StructureDefinitio...,"class,status,type",SHOULD,token,,,,Fetches a bundle of all !Encounter resources m...


In [30]:
data = [i for i in df.itertuples(index=True)]
r_type =  {d.base for d in data if '!' not in d.base}
combo_data = [i for i in df_combos.itertuples(index=True)]
search_profiles = {i.profile:i.base for i in combo_data if '!' not in i.base }

for d in data:
    print(f'Resource = {d.base}, Search Parameter = {d.code}, Exists = {d.exists}')
for c in combo_data:
    print(f'Resource = {c.base}, Combo Search Parameter = {c.combo}')

Resource = !EXAMPLE CATEGORY SEARCH, Search Parameter = category, Exists = Y
Resource = !EXAMPLE CODE SEARCH, Search Parameter = code, Exists = Y
Resource = !EXAMPLE DATE SEARCH, Search Parameter = date, Exists = Y
Resource = !EXAMPLE PATIENT SEARCH, Search Parameter = patient, Exists = Y
Resource = !EXAMPLE STATUS SEARCH, Search Parameter = status, Exists = Y
Resource = !Patient, Search Parameter = address, Exists = Y
Resource = !Patient, Search Parameter = telecom, Exists = Y
Resource = AllergyIntolerance, Search Parameter = clinical-status, Exists = Y
Resource = AllergyIntolerance, Search Parameter = patient, Exists = Y
Resource = Condition, Search Parameter = category, Exists = Y
Resource = Condition, Search Parameter = clinical-status, Exists = Y
Resource = Condition, Search Parameter = patient, Exists = Y
Resource = Encounter, Search Parameter = _id, Exists = Y
Resource = Encounter, Search Parameter = class, Exists = Y
Resource = Encounter, Search Parameter = date, Exists = Y
Res

### update core SP with additional capabiliities


- Get definitions bundle and convert to python object for ease of notation
- use sp_map to map to Type + parameter
- If need to update SP Extract the SP based on the excel file

### load SP Mapping dictionary

output shows a single SP entry for a sanity check

In [31]:
p = Path(spdef_json)
b = B.Bundle(loads(p.read_text()), strict = False)
sp = (i.resource for i in b.entry if i.resource.url == 'http://hl7.org/fhir/SearchParameter/Resource-id')
sp_for_id = next(sp)
print(dumps(sp_for_id.as_json(), indent = 4))

{
    "id": "Resource-id",
    "extension": [
        {
            "url": "http://hl7.org/fhir/StructureDefinition/structuredefinition-standards-status",
            "valueCode": "trial-use"
        }
    ],
    "base": [
        "Resource"
    ],
    "code": "_id",
    "contact": [
        {
            "telecom": [
                {
                    "system": "url",
                    "value": "http://hl7.org/fhir"
                }
            ]
        },
        {
            "telecom": [
                {
                    "system": "url",
                    "value": "http://www.hl7.org/Special/committees/fiwg/index.cfm"
                }
            ]
        }
    ],
    "date": "2019-04-18T15:23:58+00:00",
    "description": "Logical id of this artifact",
    "experimental": false,
    "expression": "Resource.id",
    "modifier": [
        "missing",
        "text",
        "not",
        "in",
        "not-in",
        "below",
        "above",
        "ofType"
    ],

## create updated SPs
sp optional 'modifier' elements are listed as comma separated list of shalls and shoulds for each:
- multipleOr
- multipleOr_conf
- multipleAnd
- multipleAnd_conf
- shall_modifier
- should_modifier
- shall_comparator
- should_comparator
- shall_chain

if not spedified then conformance is MAY 

*note that it starts out using the FHIRClient models but the switches to a dict structure to add the FHIR primitive type extensions.*

In [ ]:
def sp_expectation(conf=None):
    if not conf:
        conf = "MAY"
 
    x = X.Extension(dict(
    url = f'http://hl7.org/fhir/StructureDefinition/capabilitystatement-expectation',
    valueCode = conf
    ))
    x_dict = dict(
    extension = [x.as_json()]
    ) 
    return x_dict

def find_base_sp(d):
    if '!' not in d.base and d.update =='Y':
        #print(d.base, d.code)
        if d.code == '_id':
            sp = SP.SearchParameter(sp_for_id.as_json())
        else:
            sp = next(i.resource for i in b.entry if i.resource.code == d.code and d.base in i.resource.base)       
            sp = SP.SearchParameter(sp.as_json())
        return sp


sp_list=[]
for d in data:
        print(f'========={d.base},{d.code}==============')
        sp = find_base_sp(d)
        #print(type(sp))
        if sp:
            #print(sp.url)
            # change id and url, publisher, and contact, draft etc
            sp.id = f'{base_id.lower()}-{d.base.lower()}-{d.code.replace("_","")}'  
            sp.extension = []
            sp.derivedFrom =sp.url
            sp.url = f'{canon_base}SearchParameter/{sp.id}'
            sp.publisher = publisher
            sp.contact = [CD.ContactDetail( {"telecom" : [ publisher_endpoint ] })]
            sp.date = D.FHIRDate(f'{datetime.utcnow().isoformat()}Z')
            sp.name = pascalcase(sp.id.replace('-','_'))
            sp.name = sp.name.replace('UsCore','USCore')
            sp.status = 'active'

            sp.base = [d.base]
            
            try:
                sp.expression = [i for i in sp.expression.split('|') if i.startswith(f'{d.base}.')][0]
            except IndexError:
                try:
                    sp.expression = [i for i in sp.expression.split(' | ') if i.startswith(f'{d.base}.')][0]
                except IndexError:
                    sp.expression = f'{d.base}.id'
            try:
                sp.xpath = [i for i in sp.xpath.split('|') if i.startswith(f'f:{d.base}/')][0]
                sp.xpath = sp.xpath.strip()
            except IndexError:
                try:
                    sp.xpath = [i for i in sp.xpath.split(' | ') if i.startswith(f'f:{d.base}/')][0]
                    sp.xpath = sp.xpath.strip()
                except IndexError:
                    sp.xpath = f'{d.base}.id'
            except AttributeError:
                pass
            try:   
                sp.description = [i for i in sp.description.split('\r\n* ') if i.startswith(f'[{d.base}]')][0]
                sp.description = sp.description.split(': ')[1:]
                sp.description = ''.join(sp.description)
            except IndexError:   
                print(sp.expression)
            sp.expression = sp.expression.strip()
            sp.description = sp.description.strip() 

  
            #convert to dict since model can't handle primitive extensions
            sp_dict = sp.as_json()

            sp_dict['multipleOr'] = False if d.multipleOr in none_list else True
            sp_dict['_multipleOr'] = sp_expectation(d.multipleOr_conf)
            
            sp_dict['multipleAnd'] = False if d.multipleAnd in none_list else True
            sp_dict['_multipleAnd'] = sp_expectation(d.multipleAnd_conf)

            try:
                sp_dict['_modifier'] = []
                for m in sp_dict['modifier']: # list all modifiers in sp and assign an expectation.
                    if d.shall_modifier not in none_list and m in d.shall_modifier.split(','):
                       sp_dict['_modifier'].append(sp_expectation('SHALL'))
                    elif  d.should_modifier not in none_list and m in d.should_modifier.split(','):
                        sp_dict['_modifier'].append(sp_expectation('SHOULD'))               
                    else:
                        sp_dict['_modifier'].append(sp_expectation('MAY'))
            except KeyError:
                del(sp_dict['_modifier'])

            try:
                sp_dict['_comparator'] = []
                for m in sp_dict['comparator']: # list all comparators in sp and assign an expectation.
                   if d.shall_comparator not in none_list and m in d.shall_comparator.split(','):
                       sp_dict['_comparator'].append(sp_expectation('SHALL'))
                   elif  d.should_comparator not in none_list and m in d.should_comparator.split(','):
                        sp_dict['_comparator'].append(sp_expectation('SHOULD'))               
                   else:
                        sp_dict['_comparator'].append(sp_expectation('MAY'))
            except KeyError:
                del(sp_dict['_comparator'])

            if d.shall_chain not in none_list:
               sp_dict['chain'] = d.shall_chain.split(',')
               sp_dict['_chain'] = [sp_expectation('SHALL') for c in d.shall_chain.split(',')]

            if d.should_chain not in none_list:
               sp_dict['chain'] = d.should_chain.split(',')
               sp_dict['_chain'] = [sp_expectation('SHALL') for c in d.should_chain.split(',')]

            print(f'======================= SP = {sp_dict["id"]} =====================')
            #print(dumps(sp_dict,indent=4))
                  
            #================ add narrative =======================
            template = env.get_template(md_template[3])   
            sp_text = template.render(sp=sp_dict)
            #print(sp_text)     
            display(HTML(sp_text))
                  
            sp_dict['text'] = {}
            sp_dict['text']['status'] = "generated"
            sp_dict['text']['div'] = sp_text      
                  
            # ================ save files as resource ======================
           #save in ig_source folder
            path = Path.cwd() / ig_source_path / 'resources' / f'searchparameter-{sp_dict["id"]}.json'
            if d.base not in skip_types:    #don't write test types  
                path.write_text(dumps(sp_dict,indent=4))
            sp_list.append(sp_dict)

### Validate

In [ ]:
for i in sp_list:
    print(f'Validating {i["id"]}...........')
    r = validate(i)
    display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))               

### Create Quick Start pages using Jinja
 
- spreadsheet for sp and combos

In [32]:
search_type = dict(
    token = '{[system]}|[code]',
    id = '[id]',
    reference = '[reference]',
    string = '[string]',
    uri = '[uri]',
    date = '[date]',
    )

template = env.get_template(md_template[0])

#print(r_type)   
for profile,type in search_profiles.items():  # preprocess the for jinja templates

    sp = [d for d in data if d.base == type]
    sp_combos = [d for d in combo_data if d.profile == profile]
    #print(sp_combos[0].base)
    mods= {}
    rels = {}
    for s in sp:
            l1=s.shall_modifier.split(',') if s.shall_modifier else []
            l2=s.should_modifier.split(',') if s.should_modifier else []
            l3=s.shall_comparator.split(',') if s.shall_comparator else []
            l4=s.should_comparator.split(',') if s.should_comparator else []
            l5=s.shall_chain.split(',') if s.shall_chain else []
            l6=s.should_chain.split(',') if s.should_chain else []
            l7=s.shall_include.split(',') if s.shall_include else []
            l8=s.should_include.split(',') if s.should_include else []
            mods[s.code] = (l1+l2,l3+l4,l1,l2,l3,l4,l5,l6,l7,l8,) # 0,1 MODS AND COMPS 2,3 mods, 4.5 comps, 6,7 chains, 8,9 includes
            rels[s.code] = s.rel_url
    
    #print(mods, rels)
    

    shalls = "SHALL" in [i.base_conf for i in sp if i.display] + [i.combo_conf for i in sp_combos if i.profile == profile]  # TODO need to search both singles and combos
    shoulds = "SHOULD" in [i.base_conf for i in sp if i.display] + [i.combo_conf for i in sp_combos if i.profile == profile]
    #pprint(rels)
  
    search_md = template.render(
                    r_type=type,
                    sp=sp,
                    search_type=search_type,
                    combos=sp_combos,
                    shalls=shalls,
                    shoulds=shoulds,
                    mods = mods,
                    rels = rels,
                    )

    print(f'========={profile} {type} ==============')
    display(Markdown(search_md))
    # save
    if d.base not in skip_types:
        path = Path.cwd() / ig_source_path / 'pages' / '_includes' / f'{profile.split("/")[-1]}-search.md'
        path.write_text(search_md)

=========http://hl7.org/fhir/us/core/StructureDefinition/us-core-encounter Encounter ==============





#### Mandatory Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHALL be supported.  the  modifiers, comparators and chained parameters that are listed as optional SHOULD be supported.:


1. **SHALL** support fetching an encounter using the **[`_id`](SearchParameter-us-core-encounter-id.html)** search parameter:

    `GET [base]/Encounter[id]`

    Example:
    
    1. GET [base]/Encounter/12354
    1. GET [base]/Encounter?_id=12354

    *Implementation Notes:* Fetches a single Encounter ([how to search by the logical id] of the resource)

1. **SHALL** support searching for all encounters for a patient using the **[`patient`](SearchParameter-us-core-encounter-patient.html)** search parameter:

    `GET [base]/Encounter?patient=[reference]`

    Example:
    
    1. GET [base]/Encounter?patient=1137192

    *Implementation Notes:* Fetches a bundle of all Encounter resources for the specified patient ([how to search by reference])

1. **SHALL** support searching using the combination of the **[`date`](SearchParameter-us-core-encounter-date.html)** and **[`patient`](SearchParameter-us-core-encounter-patient.html)** search parameters:
  - including support for these comparators: `gt, lt, ge, le`

    `GET [base]/Encounter?date={gt|lt|ge|le}[date]&patient=[reference]`

    Example:
    
    1. GET [base]/Encounter?patient=example1&amp;date=ge2019

    *Implementation Notes:* Fetches a bundle of all Encounter resources matching the specified date and patient ([how to search by date] and [how to search by reference])



#### Optional Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHOULD be supported.

1. **SHOULD** support searching using the **[`identifier`](i.rel_url)** search parameter:

     `GET [base]/Encounter?identifier={[system]}|[code]`

    Example:
    
    1. GET [base]/Encounter?identifier=http://hospital.smarthealthit.org\|1032702

     *Implementation Notes:* Fetches a bundle containing any Encounter resources matching the identifier ([how to search by token])

1. **SHOULD** support searching using the combination of the **[`class`](SearchParameter-us-core-encounter-class.html)** and **[`patient`](SearchParameter-us-core-encounter-patient.html)** search parameters:

    `GET [base]/Encounter?class={[system]}|[code]&patient=[reference]`

    Example:
    
    1. GET [base]/Encounter?patient=example1&amp;class= http://terminology.hl7.org/CodeSystem/v3-ActCode code\|AMB

    *Implementation Notes:* Fetches a bundle of all Encounter resources matching the specified class and patient ([how to search by reference] and [how to search by token])

1. **SHOULD** support searching using the combination of the **[`patient`](SearchParameter-us-core-encounter-patient.html)** and **[`type`](SearchParameter-us-core-encounter-type.html)** search parameters:

    `GET [base]/Encounter?patient=[reference]&type={[system]}|[code]`

    Example:
    
    1. GET [base]/Encounter?patient=1137192&amp;type=http://www.ama-assn.org/go/cpt code\|99201

    *Implementation Notes:* Fetches a bundle of all Encounter resources matching the specified patient and type ([how to search by reference] and [how to search by token])

1. **SHOULD** support searching using the combination of the **[`patient`](SearchParameter-us-core-encounter-patient.html)** and **[`status`](SearchParameter-us-core-encounter-status.html)** search parameters:

    `GET [base]/Encounter?patient=[reference]&status={[system]}|[code]`

    Example:
    
    1. GET [base]/Encounter?patient=example1&amp;status=finished

    *Implementation Notes:* Fetches a bundle of all Encounter resources matching the specified patient and status ([how to search by reference] and [how to search by token])


{% include link-list.md %}

=========http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient Patient ==============





#### Mandatory Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHALL be supported.  the  modifiers, comparators and chained parameters that are listed as optional SHOULD be supported.:


1. **SHALL** support fetching a Patient using the **[`_id`](SearchParameter-us-core-patient-id.html)** search parameter:

    `GET [base]/Patient[id]`

    Example:
    
    1. GET [base]/Patient/1032702
    1. GET [base]/Patient?_id=1032702

    *Implementation Notes:*  ([how to search by the logical id] of the resource)

1. **SHALL** support searching a patient by an identifier such as a MPI using the **[`identifier`](SearchParameter-us-core-patient-identifier.html)** search parameter:

    `GET [base]/Patient?identifier={[system]}|[code]`

    Example:
    
    1. GET [base]/Patient?identifier=http://hospital.smarthealthit.org\|1032702

    *Implementation Notes:* Fetches a bundle containing any Patient resources matching the identifier ([how to search by token])

1. **SHALL** support searching for a patient by a string match of any part of name using the **[`name`](SearchParameter-us-core-patient-name.html)** search parameter:

    `GET [base]/Patient?name=[string]`

    Example:
    
    1. GET [base]/Patient?name=Shaw

    *Implementation Notes:* Fetches a bundle of all Patient resources matching the name ([how to search by string])

1. **SHALL** support searching using the combination of the **[`birthdate`](SearchParameter-us-core-patient-birthdate.html)** and **[`name`](SearchParameter-us-core-patient-name.html)** search parameters:

    `GET [base]/Patient?birthdate=[date]&name=[string]`

    Example:
    
    1. GET [base]/Patient?name=Shaw&amp;birthdate=2007-03-20

    *Implementation Notes:* Fetches a bundle of all Patient resources matching the specified birthdate and name ([how to search by date] and [how to search by string])

1. **SHALL** support searching using the combination of the **[`gender`](SearchParameter-us-core-patient-gender.html)** and **[`name`](SearchParameter-us-core-patient-name.html)** search parameters:

    `GET [base]/Patient?gender={[system]}|[code]&name=[string]`

    Example:
    
    1. GET [base]/Patient?name=Shaw&amp;gender=female

    *Implementation Notes:* Fetches a bundle of all Patient resources matching the specified gender and name ([how to search by string] and [how to search by token])



#### Optional Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHOULD be supported.

1. **SHOULD** support searching using the combination of the **[`birthdate`](SearchParameter-us-core-patient-birthdate.html)** and **[`family`](SearchParameter-us-core-patient-family.html)** search parameters:

    `GET [base]/Patient?birthdate=[date]&family=[string]`

    Example:
    
    1. GET [base]/Patient?family=Shaw&amp;birthdate=2007-03-20

    *Implementation Notes:* Fetches a bundle of all Patient resources matching the specified birthdate and family ([how to search by date] and [how to search by string])

1. **SHOULD** support searching using the combination of the **[`family`](SearchParameter-us-core-patient-family.html)** and **[`gender`](SearchParameter-us-core-patient-gender.html)** search parameters:

    `GET [base]/Patient?family=[string]&gender={[system]}|[code]`

    Example:
    
    1. GET [base]/Patient?family=Shaw&amp;gender=female

    *Implementation Notes:* Fetches a bundle of all Patient resources matching the specified family and gender ([how to search by string] and [how to search by token])


{% include link-list.md %}

=========http://hl7.org/fhir/us/core/StructureDefinition/us-core-condition Condition ==============


...




#### Mandatory Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHALL be supported.  the  modifiers, comparators and chained parameters that are listed as optional SHOULD be supported.:


1. **SHALL** support searching using the combination of the **[`patient`](SearchParameter-us-core-observation-patient.html)** and **[`code`](SearchParameter-us-core-observation-code.html)** search parameters:

    `GET [base]/Observation?patient=[reference]&code=http://loinc.org|72166-2`

    Example:
    
    1. GET [base]/Observation?patient=1032702&amp;code=http://loinc.org\|72166-2

    *Implementation Notes:* Fetches a bundle of all Observation resources for the specified patient and observation code. ([how to search by reference] and [how to search by token])





{% include link-list.md %}

=========http://hl7.org/fhir/us/core/StructureDefinition/us-core-vitalsigns Observation ==============





#### Mandatory Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHALL be supported.  the  modifiers, comparators and chained parameters that are listed as optional SHOULD be supported.:


1. **SHALL** support searching using the combination of the **[`patient`](SearchParameter-us-core-observation-patient.html)** and **[`category`](SearchParameter-us-core-observation-category.html)** search parameters:

    `GET [base]/Observation?patient=[reference]&category=http://hl7.org/fhir/observation-category|vital-signs`

    Example:
    
    1. GET [base]/Observation?patient=1134281&amp;category=http://hl7.org/fhir/observation-category\|vital-signs

    *Implementation Notes:* Fetches a bundle of all Observation resources for the specified patient and a category code = `vital-signs` ([how to search by reference] and [how to search by token])

1. **SHALL** support searching using the combination of the **[`patient`](SearchParameter-us-core-observation-patient.html)** and **[`code`](SearchParameter-us-core-observation-code.html)** search parameters:

    `GET [base]/Observation?patient=[reference]&code={[system]}|[code]`

    Example:
    
    1. GET [base]/Observation?patient=1186747&amp;code=http://loinc.org\|8867-4,http://loinc.org\|9279-1,http://loinc.org\|85354-9

    *Implementation Notes:* Fetches a bundle of all Observation resources for the specified patient and observation code(s).  SHOULD support search by multiple codes. The Observation `code` parameter searches `Observation.code only. ([how to search by reference] and [how to search by token])

1. **SHALL** support searching using the combination of the **[`patient`](SearchParameter-us-core-observation-patient.html)** and **[`category`](SearchParameter-us-core-observation-category.html)** and **[`date`](SearchParameter-us-core-observation-date.html)** search parameters:
  - including support for these comparators: `gt, lt, ge, le`

    `GET [base]/Observation?patient=[reference]&category=http://hl7.org/fhir/observation-category|vital-signs&date={gt|lt|ge|le}[date]`

    Example:
    
    1. GET [base]/Observation?patient=1134281&amp;category=http://hl7.org/fhir/observation-category\|vital-signs&amp;date=ge2019

    *Implementation Notes:* Fetches a bundle of all Observation resources for the specified patient and date and a category code = `vital-signs` ([how to search by reference] and [how to search by token] and [how to search by date])



#### Optional Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHOULD be supported.

1. **SHOULD** support searching using the combination of the **[`patient`](SearchParameter-us-core-observation-patient.html)** and **[`category`](SearchParameter-us-core-observation-category.html)** and **[`status`](SearchParameter-us-core-observation-status.html)** search parameters:

    `GET [base]/Observation?patient=[reference]&category={[system]}|[code]&status={[system]}|[code]`

    Example:
    
    1. GET [base]/Observation?patient=1134281&amp;category=http://hl7.org/fhir/observation-category\|vital-signs&amp;status=final

    *Implementation Notes:* Fetches a bundle of all Observation resources for the specified patient and category and status ([how to search by reference] and [how to search by token])

1. **SHOULD** support searching using the combination of the **[`patient`](SearchParameter-us-core-observation-patient.html)** and **[`code`](SearchParameter-us-core-observation-code.html)** and **[`date`](SearchParameter-us-core-observation-date.html)** search parameters:
  - including support for these comparators: `gt, lt, ge, le`

    `GET [base]/Observation?patient=[reference]&code={[system]}|[code]&date={gt|lt|ge|le}[date]`

    Example:
    
    1. GET [base]/Observation?patient=1186747&amp;code=http://loinc.org\|8867-4,http://loinc.org\|9279-1,http://loinc.org\|85354-9&amp;date=ge2019

    *Implementation Notes:* Fetches a bundle of all Observation resources for the specified patient and date and report code(s).  SHOULD support search by multiple codes. ([how to search by reference] and [how to search by token] and [how to search by date])


{% include link-list.md %}

=========http://hl7.org/fhir/us/core/StructureDefinition/us-core-documentreference DocumentReference ==============





#### Mandatory Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHALL be supported.  the  modifiers, comparators and chained parameters that are listed as optional SHOULD be supported.:


1. **SHALL** support fetching a DocumentReference using the **[`_id`](SearchParameter-us-core-documentreference-_id.html)** search parameter:

    `GET [base]/DocumentReference[id]`

    Example:
    
    1. GET [base]/DocumentReference/2169591
    1. GET [base]/DocumentReference?_id=2169591

    *Implementation Notes:* Fetches a single DocumentReference. The document itself is represented as a base64 encoded binary data element or retrieved using the link provided by the resource. If the document is a  relative link to a [Binary] resource like a resource reference, it can be subsequently retrieved using: `GET [base]/Binary/[id]`. ([how to search by the logical id] of the resource)

1. **SHALL** support searching for all documentreferences for a patient using the **[`patient`](SearchParameter-us-core-documentreference-patient.html)** search parameter:

    `GET [base]/DocumentReference?patient=[reference]`

    Example:
    
    1. GET [base]/DocumentReference?patient=1137192

    *Implementation Notes:* Fetches a bundle of all DocumentReference resources for the specified patient. See the implementation notes above for how to access the actual document. ([how to search by reference])

1. **SHALL** support searching using the combination of the **[`patient`](SearchParameter-us-core-documentreference-patient.html)** and **[`category`](SearchParameter-us-core-documentreference-category.html)** search parameters:

    `GET [base]/DocumentReference?patient=[reference]&category=http://hl7.org/fhir/us/core/CodeSystem/us-core-documentreference-category|clinical-note`

    Example:
    
    1. GET [base]/DocumentReference?patient=1235541&amp;category=http://hl7.org/fhir/us/core/CodeSystem/us-core-documentreference-category\|clinical-note

    *Implementation Notes:* Fetches a bundle of all DocumentReference resources for the specified patient and category = `clinical-note`.  See the implementation notes above for how to access the actual document. ([how to search by reference] and [how to search by token])

1. **SHALL** support searching using the combination of the **[`patient`](SearchParameter-us-core-documentreference-patient.html)** and **[`category`](SearchParameter-us-core-documentreference-category.html)** and **[`date`](SearchParameter-us-core-documentreference-date.html)** search parameters:
  - including support for these comparators: `gt, lt, ge, le`

    `GET [base]/DocumentReference?patient=[reference]&category=http://hl7.org/fhir/us/core/CodeSystem/us-core-documentreference-category|clinical-note&date={gt|lt|ge|le}[date]`

    Example:
    
    1. GET [base]/DocumentReference?patient=1235541&amp;category=http://hl7.org/fhir/us/core/CodeSystem/us-core-documentreference-category\|clinical-note&amp;date=ge2019

    *Implementation Notes:* Fetches a bundle of all DocumentReference resources for the specified patient and category = `clinical=note` and date. See the implementation notes above for how to access the actual document. ([how to search by reference] and [how to search by token] and [how to search by date])

1. **SHALL** support searching using the combination of the **[`patient`](SearchParameter-us-core-documentreference-patient.html)** and **[`type`](SearchParameter-us-core-documentreference-type.html)** search parameters:

    `GET [base]/DocumentReference?patient=[reference]&type={[system]}|[code]`

    Example:
    
    1. GET [base]/DocumentReference?patient=1316024&amp;type=http://loinc.org 18842-5

    *Implementation Notes:* Fetches a bundle of all DocumentReference resources for the specified patient and type. See the implementation notes above for how to access the actual document. ([how to search by reference] and [how to search by token])



#### Optional Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHOULD be supported.

1. **SHOULD** support searching using the combination of the **[`patient`](SearchParameter-us-core-documentreference-patient.html)** and **[`status`](SearchParameter-us-core-documentreference-status.html)** search parameters:

    `GET [base]/DocumentReference?patient=[reference]&status={[system]}|[code]`

    Example:
    
    1. GET [base]/DocumentReference?patient=1235541

    *Implementation Notes:* Fetches a bundle of all DocumentReference resources for the specified patient and status. See the implementation notes above for how to access the actual document. ([how to search by reference] and [how to search by token])

1. **SHOULD** support searching using the combination of the **[`patient`](SearchParameter-us-core-documentreference-patient.html)** and **[`type`](SearchParameter-us-core-documentreference-type.html)** and **[`period`](SearchParameter-us-core-documentreference-period.html)** search parameters:
  - including support for these comparators: `gt, lt, ge, le`

    `GET [base]/DocumentReference?patient=[reference]&type={[system]}|[code]&period={gt|lt|ge|le}[date]`

    Example:
    
    1. GET [base]/DocumentReference?patient=2169591&amp;type=34133-9&amp;period=ge2019

    *Implementation Notes:* Fetches a bundle of all DocumentReference resources for the specified patient and type and period. See the implementation notes above for how to access the actual document. ([how to search by reference] and [how to search by token] and [how to search by date])


{% include link-list.md %}

### Create Markdown Text for SearchParameters Page

- Using Jinja2 Template create markdown file for searchparameters page
- Use spreadsheet as source

In [ ]:
for d in data:
    if '!' in d.base:
        print(d.base,True)

# get id for each row in spreadsheet:
r_list={ d.base for d in data if '!' not in d.base and d.base not in skip_types}
d_tup = []
for d in data:
    if d.base in r_list:
        sp_id= [sp['id'] for sp in sp_list if d.base in sp['base'] and sp['code'] in [d.code,'id'] ]
        d_tup.append((sp_id[0] if sp_id else None,d))

template = env.get_template(md_template[1])   

searchparameters_md = template.render(sp_list=d_tup,r_list=r_list)

display(Markdown(searchparameters_md))
# save in pages folder
path = Path.cwd() / ig_source_path / 'pages' / 'searchparameters.md'
path.write_text(searchparameters_md)